In [1]:
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
from gensim.models import KeyedVectors


In [2]:
!pip install tweet-preprocessor

In [3]:
import pandas as pd
import re
import preprocessor as p
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [4]:
import pandas as pd

#creating a dataframe for our dataset "subject_questions"
df = pd.read_csv("subjects-questions.csv")
df=df.head(4000)
#printing first 5 rows of our dataset/dataframe
df.head()

,eng,Subject
0,An anti-forest measure is\nA. Afforestation\nB...,Biology
1,"Among the following organic acids, the acid pr...",Chemistry
2,If the area of two similar triangles are equal...,Maths
3,"In recent year, there has been a growing\nconc...",Biology
4,Which of the following statement\nregarding tr...,Physics


In [5]:
df.shape

(4000, 2)

In [6]:
unique_values = df['Subject'].unique()
print(unique_values)

['Biology' 'Chemistry' 'Maths' 'Physics']


In [7]:
#encoding labels

df['Subject']=df['Subject'].map({'Biology':0,
                             'Chemistry':1,
                             'Physics':2,
                                 'Maths':3})

In [8]:
df.head()

,eng,Subject
0,An anti-forest measure is\nA. Afforestation\nB...,0
1,"Among the following organic acids, the acid pr...",1
2,If the area of two similar triangles are equal...,3
3,"In recent year, there has been a growing\nconc...",0
4,Which of the following statement\nregarding tr...,2


In [9]:
def clean_tweets(df):
    # punctuations we want to be replaced
    REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\|)|(\()|(\))|(\[)|(\])|(\%)|(\$)|(\>)|(\<)|(\{)|(\})")
    REPLACE_WITH_SPACE = re.compile("(<br\s/><br\s/?)|(-)|(/)|(:).")
    corpus = []
    for line in df:
        # clean data by sending it to tweet_processor
        sentence = p.clean(line)
        # remove puctuation
        sentence = REPLACE_NO_SPACE.sub("", sentence.lower()) # convert all tweets to lower cases
        sentence = REPLACE_WITH_SPACE.sub(" ", sentence)
        corpus.append(sentence)
    return corpus

In [10]:
df['eng_clean'] = clean_tweets(df['eng'])
df.shape

(4000, 3)

In [11]:
df['eng_clean']

0       an anti forest measure isa afforestationb sele...
1       among the following organic acids the acid pre...
2       if the area of two similar triangles are equal...
3       in recent year there has been a growingconcern...
4       which of the following statementregarding tran...
                              ...                        
3995    the distance between the ends of thewings of a...
3996    the radius of \ h e^+ \ ion is \ x \dota \ in ...
3997    one oscillation completed by a vibrating body ...
3998    three resistances of \ \omega \omega \ and \ \...
3999    a block of mass \ m \ moving at a speed \ v \c...
Name: eng_clean, Length: 4000, dtype: object

In [12]:
corpus = list(df['eng_clean'])

In [13]:
corpus

['an anti forest measure isa afforestationb selective grazingc  clearing forestd selective felling',
 'among the following organic acids the acid present in rancid butter isa  pyruvic acidb lactic acidc butyric acidd acetic acid',
 'if the area of two similar triangles are equal then they area  equilateralb isoscelesc congruentd not congruent',
 'in recent year there has been a growingconcern about the gradually increasing average global temperatures if this trend continues would you expert the distributional range of some species to be affected',
 'which of the following statementregarding transformer is incorrecta a transformer makes use of faradays lawb the ferromagnetic properties of an iron core is used by it to efficiently raise or lower ac voltagec it cannot increase power so that if the voltage is raised the current is proportionally lowered and vice versad the current induced in primary circuit is of same nature as in secondary circuit with differences in amplitude only',
 'fe

In [14]:
!pip install fasttext


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4479739 sha256=4c26b1b424d27db7dd3e6222c338bda4fbd90ac6c1012a317a32aae58efef581
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [15]:
import fasttext.util
fasttext.util.download_model('en', if_exists='ignore')  # English
ft = fasttext.load_model('cc.en.300.bin')

 (93.49%) [

KeyboardInterrupt: ignored

In [ ]:
# Word2Vec
from gensim.models import Word2Vec

word2vec_model = Word2Vec([ft.get_word_vector(word) for word in ft.words])

# GloVe
#glove_model = KeyedVectors.load_word2vec_format('path_to_glove_model', binary=False)


In [ ]:
corpus_embeddings = []
for sentence in corpus:
    words = sentence.split()
    sentence_vector = np.mean([word2vec_model.wv[word] for word in words], axis=0)
    # For GloVe, replace word2vec_model.wv[word] with glove_model[word]
    corpus_embeddings.append(sentence_vector)


In [ ]:
num_clusters = 4
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_
cluster_assignment

In [ ]:
df['cluster']=cluster_assignment
df.head(20)

In [ ]:
#label_mapping={0:0,2:1,3:2,1:3}

In [ ]:
#df['cluster']=df['cluster'].map(label_mapping)


In [ ]:
import numpy as np

def tri_et_index(tableau):
    # Obtenir les indices des valeurs triées en ordre décroissant
    indices_tries = np.flip(np.argsort(tableau))

    return indices_tries

In [ ]:
import numpy as np

def map_labels(predicted_labels, true_labels):
    unique_predicted_labels = np.unique(predicted_labels)
    label_mapping = {index: 10 for index in unique_predicted_labels}

    for predicted_label in unique_predicted_labels:
        i=0
        mask = predicted_labels == predicted_label
        cluster_true_labels = true_labels[mask]
        true_label_counts = np.bincount(cluster_true_labels)
        indexs=tri_et_index(true_label_counts)
        most_frequent_true_label=indexs[i]
        l=[label_mapping[label] for label in unique_predicted_labels]

        while most_frequent_true_label in l[:predicted_label]:
            most_frequent_true_label = indexs[i+1]
            i+=1
        label_mapping[predicted_label] = most_frequent_true_label

    return label_mapping

label_mapping = map_labels(cluster_assignment,df["Subject"])


In [ ]:
mapped_predicted_labels = np.array([label_mapping[label] for label in cluster_assignment])
unique_values = np.unique(mapped_predicted_labels)
label_mapping

In [ ]:
from sklearn.metrics import accuracy_score
clustering_accuracy = accuracy_score(df["Subject"], mapped_predicted_labels)

clustering_accuracy